In [ ]:
from enum import Enum
import sys
sys.path.append('../src')
from pathlib import Path


from bokeh.models import ColumnDataSource, ColorBar
from bokeh.transform import linear_cmap
from bokeh.models import LinearColorMapper
from bokeh.palettes import Viridis256 as palette
from bokeh.plotting import figure, output_notebook, show
import numpy as np
import pandas as pd

from nlkda.models.utils import ModelEnum
from nlkda.utils import MLFlowClient
from nlkda.settings import K_MAX
from nlkda.data.loader import get_data, DatasetEnum

output_notebook()

data_root = Path("/tmp/data/")
exp = data_root / 'experiments'
db = MLFlowClient(root=str(exp), tracking_uri="localhost:5000/")

In [ ]:
def plot_k_dist(
    df,
    max_val=None,
    min_val=None,
):
    color_mapper = LinearColorMapper(palette=palette)
    
    if max_val is None:
        max_val = max(df['kdist'])
    if min_val is None:
        min_val = min(df['kdist'])
  
    mapper = linear_cmap(field_name='kdist', palette=palette, low=min_val, high=max_val)
    source = ColumnDataSource(dict(index=df['p_index'],x=df['x'],y=df['y'],kdist=df['kdist']))

    p = figure(
        x_axis_location=None, 
        y_axis_location=None,
        plot_width=500,
        plot_height=500,
        tools="pan,wheel_zoom,reset,hover,save"
    )

    p.grid.grid_line_color = None

    p.circle('x', 'y', source=source,
              fill_color={'field': 'kdist', 'transform': color_mapper},
              fill_alpha=1, line_color="white", line_width=0.1, size=8)

    color_bar = ColorBar(color_mapper=mapper['transform'], width=12,  location=(0,0))
    p.add_layout(color_bar, 'right')
    show(p)

In [ ]:
k = 256
ds_enum = DatasetEnum.OL
x, y, dist = get_data(ds_enum, data_root / ds_enum.value)

df = pd.DataFrame(data=x, columns = ['x', 'y'])
df['p_index'] = np.arange(len(df))
df['kdist'] = y[:,k-1]

plot_k_dist(df)